In [1]:
import os
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from watermark import watermark

%load_ext watermark

In [2]:
%watermark --iversions

selenium : 4.18.1
bs4      : 4.12.3
watermark: 2.5.0
pandas   : 2.2.2



In [3]:
print(watermark())

Last updated: 2024-11-16T11:26:47.487710-03:00

Python implementation: CPython
Python version       : 3.11.10
IPython version      : 8.29.0

Compiler    : MSC v.1941 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [4]:
# Função para extrair páginas e salvar seu conteúdo HTML
def scrape_pages(base_url, num_pages):
    
    # Configura o driver do navegador (Firefox)
    driver = webdriver.Firefox()

    # Cria pasta 'dados' caso ainda não exista
    if not os.path.exists('dados'):
        os.makedirs('dados')

    try:
        for page in range(1, num_pages + 1):
            
            # Constrói URL para a página atual
            url = f'{base_url}{page}'
            
            # Acessa a página desejada
            driver.get(url)

            # Espera um pouco para a página carregar completamente
            time.sleep(3)

            # Obtém o conteúdo HTML da página
            page_source = driver.page_source

            # Salva o conteúdo em um arquivo HTML na pasta 'dados'
            with open(f'dados/pagina_{page}.html', 'w', encoding='utf-8') as f:
                f.write(page_source)

            # Imprime uma mensagem para cada página salva
            print(f'Salvo: dados/pagina_{page}.html')

    finally:
        # Fecha o navegador
        driver.quit()

# URL base para extração das páginas
base_url = 'https://www.mpsp.mp.br/promotorias-de-justica-resultados?cityId=0&activityAreaId=1&page='

# Número de páginas paraa acessar
num_pages = 57

# Executa a função para iniciar a extração
scrape_pages(base_url, num_pages)


Salvo: dados/pagina_1.html
Salvo: dados/pagina_2.html
Salvo: dados/pagina_3.html
Salvo: dados/pagina_4.html
Salvo: dados/pagina_5.html
Salvo: dados/pagina_6.html
Salvo: dados/pagina_7.html
Salvo: dados/pagina_8.html
Salvo: dados/pagina_9.html
Salvo: dados/pagina_10.html
Salvo: dados/pagina_11.html
Salvo: dados/pagina_12.html
Salvo: dados/pagina_13.html
Salvo: dados/pagina_14.html
Salvo: dados/pagina_15.html
Salvo: dados/pagina_16.html
Salvo: dados/pagina_17.html
Salvo: dados/pagina_18.html
Salvo: dados/pagina_19.html
Salvo: dados/pagina_20.html
Salvo: dados/pagina_21.html
Salvo: dados/pagina_22.html
Salvo: dados/pagina_23.html
Salvo: dados/pagina_24.html
Salvo: dados/pagina_25.html
Salvo: dados/pagina_26.html
Salvo: dados/pagina_27.html
Salvo: dados/pagina_28.html
Salvo: dados/pagina_29.html
Salvo: dados/pagina_30.html
Salvo: dados/pagina_31.html
Salvo: dados/pagina_32.html
Salvo: dados/pagina_33.html
Salvo: dados/pagina_34.html
Salvo: dados/pagina_35.html
Salvo: dados/pagina_36.html
S

In [5]:
def maps_informations(div_get):

  # Lista vazia para armazenar endereços das promotorias contidas na página HTML
  dados_promotorias = []

  # Encontra divisórias com as informações das promotorias na página HTML
  divs_info = soup.find_all('div', class_='accordion-map')

  # Itera sobre o conteúdo das divisórias de interesse
  for div in divs_info:
      
    # Extrai os dados e adiciona-os à lista criada
    dados = div.get(div_get)
    dados_promotorias.append(dados)

  # Retorna lista com as informações das promotorias
  return dados_promotorias


In [6]:
# Cria lista de nomes dos arquivos com extensão .html
n_files = range(2, 57 + 1)
filepaths = ['dados/pagina_1.html']

for n in n_files:
  filepaths.append(f'dados/pagina_{n}.html')


In [7]:
# Cria lista vazia para armazenar DataFrames
dataframes = []

# Itera sobre os caminhos dos arquivos
for filepath_index, filepath in enumerate(filepaths):
    with open(filepath, 'r', encoding='utf-8') as page:
        html = page.read()  # Texto html
        soup = BeautifulSoup(html, 'html.parser')

    # Lista vazia para armazenar os nomes das promotorias
    nomes = []

    # Encontra divisórias com nomes das promotorias
    nomes_promotorias_divs = soup.find_all('div', class_='accordion-title-adjust-item pr-lg-5')

    # Itera sobre o conteúdo das divisórias
    for div in nomes_promotorias_divs:
        conteudos = div.find_all('p')
        if conteudos:
            nomes.append(conteudos[0].text.strip())

    # Coleta informações de endereços, latitudes e longitudes
    atributos = ['data-address', 'data-latitude', 'data-longitude']
    informacoes = {attr: maps_informations(div_get=attr) for attr in atributos}

    # Verifica se todas as listas têm o mesmo comprimento
    if len(nomes) == len(informacoes['data-address']) == len(informacoes['data-latitude']) == len(informacoes['data-longitude']):
        
        # Transforma listas em dicionário e constrói DataFrame
        promotorias_dict = {
            'PJ': nomes,
            'ENDEREÇO': informacoes['data-address'],
            'LATITUDE': informacoes['data-latitude'],
            'LONGITUDE': informacoes['data-longitude']
        }
        promotorias_df = pd.DataFrame(promotorias_dict)
        dataframes.append(promotorias_df)
    else:
        print(f'Erro: Tamanhos diferentes em {filepath}')

# Concatena todos os DataFrames em um único DataFrame
promotorias_mpsp = pd.concat(dataframes, ignore_index=True)

# Exibe informações sobre o DataFrame
print(f'Linhas: {promotorias_mpsp.shape[0]} | Colunas: {promotorias_mpsp.shape[1]}')

# Exibe 5 primeiras linhas do DataFrame
promotorias_mpsp.head()


Linhas: 340 | Colunas: 4


,PJ,ENDEREÇO,LATITUDE,LONGITUDE
0,PROMOTORIA DE JUSTIÇA DE ANDRADINA,"Rua Amazonas, 808 - Andradina - SP",-20.890823448799644,-51.3749834875957
1,PROMOTORIA DE JUSTIÇA DE ARAÇATUBA,"Avenida Joaquim Pompeu de Toledo, 1261 - Araça...",-21.22269501035656,-50.4430001736112
2,PROMOTORIA DE JUSTIÇA DE BILAC,"Rua Olavo Bilac, 466 - Bilac - SP",-21.405886718020316,-50.473083472654906
3,PROMOTORIA DE JUSTIÇA DE BIRIGUI,"Rua Francisco Martins Archila, 232 - Birigui - SP",-21.30675323983098,-50.33996844578154
4,PROMOTORIA DE JUSTIÇA DE BURITAMA,"Rua Joaquim Pereira Rosa, 607, Gleba 02, Burit...",-21.069653773376974,-50.1485988179285


In [8]:
# Identifica Promotoria(s) com formato incorreto de Longitude (s/ sinal negativo)
longitudes_incorretas = promotorias_mpsp[(promotorias_mpsp['LONGITUDE'].str[0] != '-')]
idx_longitude_incorreta = longitudes_incorretas.index[0]

# Exibe linha com Longitude incorreta
linha_longitude_incorreta = promotorias_mpsp.iloc[idx_longitude_incorreta]
print('Longitude incorreta:')
print(linha_longitude_incorreta)

# Corrige valores de Longitude
promotorias_mpsp.loc[idx_longitude_incorreta, 'LONGITUDE'] = '-46.98278360804578'

# Confirma a correção
print('\nLongitude corrigida:')
print(promotorias_mpsp.iloc[idx_longitude_incorreta])


Longitude incorreta:
PJ                  PROMOTORIA DE JUSTIÇA DE VINHEDO
ENDEREÇO     Rua dos Pintassilgos, 59 - Vinhedo - SP
LATITUDE                          -23.03664580236691
LONGITUDE                          46.98278360804578
Name: 85, dtype: object

Longitude corrigida:
PJ                  PROMOTORIA DE JUSTIÇA DE VINHEDO
ENDEREÇO     Rua dos Pintassilgos, 59 - Vinhedo - SP
LATITUDE                          -23.03664580236691
LONGITUDE                         -46.98278360804578
Name: 85, dtype: object


In [9]:
# Identifica Promotoria(s) com formato incorreto de coordenadas (em que o 
# quarto caractere das strings de latitude e longitude não é um ponto)
coordenadas_incorretas = promotorias_mpsp[
    (promotorias_mpsp['LATITUDE'].str[3] != '.') &
    (promotorias_mpsp['LONGITUDE'].str[3] != '.')
]

idx_coordendas_incorretas = coordenadas_incorretas.index[0]

# Exibe linha com coordenadas incorretas
linha_coordenadas_incorretas = promotorias_mpsp.iloc[idx_coordendas_incorretas]
print('Coordenadas incorretas:')
print(linha_coordenadas_incorretas)

# Corrige valores das coordenadas
promotorias_mpsp.loc[idx_coordendas_incorretas, 'LATITUDE'] = '-22.992981159412644'
promotorias_mpsp.loc[idx_coordendas_incorretas, 'LONGITUDE'] = '-49.86517911406816'

# Confirma a correção
print('\nCoordenadas corrigidas:')
print(promotorias_mpsp.iloc[idx_coordendas_incorretas])


Coordenadas incorretas:
PJ               PROMOTORIA DE JUSTIÇA DE OURINHOS - CÍVEL
ENDEREÇO     Rua dos Expedicionários, 1895 - Ourinhos - SP
LATITUDE                            -2.2992981159412644E16
LONGITUDE                            -4.986517911406816E15
Name: 38, dtype: object

Coordenadas corrigidas:
PJ               PROMOTORIA DE JUSTIÇA DE OURINHOS - CÍVEL
ENDEREÇO     Rua dos Expedicionários, 1895 - Ourinhos - SP
LATITUDE                               -22.992981159412644
LONGITUDE                               -49.86517911406816
Name: 38, dtype: object


In [10]:
# Salva como arquivo CSV
promotorias_mpsp[['PJ', 'LATITUDE', 'LONGITUDE']].to_csv('coordenadas_promotorias_mpsp.csv', encoding='utf-8', index=False)